In [ ]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
from music21 import note, chord, scale, stream, interval
import random

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

TICKS_PER_BEAT = 480  # Standard MIDI resolution
TEMPO_BPM = 120
OUTPUT_FILE = "my_composition.mid"

In [ ]:
# =============================================================================
# MUSICAL TIME HELPERS
# =============================================================================

class MusicalTime:
    """Convert musical durations to MIDI ticks"""

    def __init__(self, ticks_per_beat=480):
        self.tpb = ticks_per_beat

    def whole(self): return self.tpb * 4
    def half(self): return self.tpb * 2
    def quarter(self): return self.tpb
    def eighth(self): return self.tpb // 2
    def sixteenth(self): return self.tpb // 4
    def dotted(self, duration): return int(duration * 1.5)
    def triplet(self, duration): return int(duration * 2 / 3)


mt = MusicalTime(TICKS_PER_BEAT)


# =============================================================================
# MUSIC21 HELPERS - Music Theory Operations
# =============================================================================

def get_scale_notes(root='C', octave=4, scale_type='major'):
    """
    Get notes from a scale using music21

    Args:
        root: Root note (C, D, E, etc.)
        octave: Octave number
        scale_type: 'major', 'minor', 'dorian', etc.

    Returns:
        List of MIDI note numbers
    """
    if scale_type == 'major':
        s = scale.MajorScale(f'{root}{octave}')
    elif scale_type == 'minor':
        s = scale.MinorScale(f'{root}{octave}')
    elif scale_type == 'dorian':
        s = scale.DorianScale(f'{root}{octave}')
    else:
        s = scale.MajorScale(f'{root}{octave}')

    # Get one octave of pitches
    pitches = s.getPitches(f'{root}{octave}', f'{root}{octave + 1}')

    # Convert to MIDI note numbers
    return [p.midi for p in pitches]


def get_chord_notes(chord_symbol, octave=4):
    """
    Get notes from a chord using music21

    Args:
        chord_symbol: 'C', 'Dm', 'G7', 'Am7', etc.
        octave: Base octave

    Returns:
        List of MIDI note numbers
    """
    # Parse chord symbol
    c = chord.Chord(f'{chord_symbol}{octave}')
    return [p.midi for p in c.pitches]


def transpose_notes(notes, semitones):
    """Transpose a list of MIDI notes by semitones"""
    return [n + semitones for n in notes]


def get_interval_from_note(note_num, interval_name):
    """
    Get a note at a specific interval from a given note

    Args:
        note_num: MIDI note number
        interval_name: 'P5' (perfect 5th), 'M3' (major 3rd), etc.

    Returns:
        MIDI note number
    """
    n = note.Note()
    n.midi = note_num

    i = interval.Interval(interval_name)
    transposed = n.transpose(i)

    return transposed.midi

